Доброго времени суток!

Для начала напишем class MusicFinder, который сможет распарсить сообщение пользователя и грамотно запомнить url

In [ ]:
from bs4 import BeautifulSoup
import requests
from gtts import gTTS

class MusicFinder:
    def __init__(self,
                 song_name: str):
        singer, song = song_name.split("/")
        singer = singer.lower()
        singer = singer.replace(" ", "_")
        song = song.lower()
        song = song.replace(" ", "_")
        self._song_name = song_name
        self._singer = singer
        self._song = song
        self._url = self._singer[0] + "/" + self._singer + "/" + self._song

Дальше напишем для него две функции - найти текст песни и найти перевод песни

In [ ]:
    def find_original_text(self) -> str:
        url = f'https://en.lyrsense.com/{self._singer}/{self._song}'
        response = requests.get(url)

        if response.status_code != 200:
            return "Приношу свои извинения, я не смог найти данную песню"

        soup = BeautifulSoup(response.text, 'lxml')
        types_1 = soup.find_all("span", "puzEng")
        final_answer = ""
        for i in types_1:
            final_answer += i.text
            final_answer += "\n"
        return final_answer

In [ ]:
    def find_translation(self) -> str:
        url = f'https://en.lyrsense.com/{self._singer}/{self._song}'
        response = requests.get(url)

        if response.status_code != 200:
            return "Приношу свои извинения, я не смог найти данную песню"

        soup = BeautifulSoup(response.text, 'lxml')
        types_1 = soup.find_all("p", id == "ru_text")[1]
        final_answer = ""
        for i in types_1:
            final_answer += i.text
            final_answer += "\n"
        return final_answer


Пишем функцию для озвучки английского текста

In [ ]:
    def dub_song(self):
        txt = self.find_original_text()
        myobj = gTTS(text=txt, lang="en", slow=False)
        myobj.save('bad_words_in_strange_order.mp3')

Дальше у нас идет вторая часть проекта - написание телеграм бота, который будет обрабатывать запросы

In [ ]:
import telebot
from telebot import types


bot = telebot.TeleBot("1824128367:AAEThtMft1hFinf7j74GqiSgRhERwP142ZY") # подключаем бота

Прописываем что он будет делать при команде /start и /help - выдавать клавиатуру с опциями и давать описание

In [ ]:
@bot.message_handler(commands=['start', 'help'])
def send_keyboard(message, text="Привет, я музбот! Я умею находить слова английских песен и их переводы! "
                                "Давай Начнем?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('Найти текст песни')
    itembtn2 = types.KeyboardButton('Найти перевод песни')
    itembtn3 = types.KeyboardButton('Озвучь песню')
    itembtn4 = types.KeyboardButton('Заканчиваем')
    keyboard.add(itembtn1, itembtn2)
    keyboard.add(itembtn3, itembtn4)
    msg = bot.send_message(message.from_user.id,
                           text=text, reply_markup=keyboard)
    bot.register_next_step_handler(msg, callback_worker)


Прописываем callback_worker, который будет обрабатывать сообщения и перенаправлять их 

In [ ]:
def callback_worker(call):
    if call.text == "Найти текст песни":
        msg = bot.send_message(call.chat.id,
                               'Давайте найдем текст песни! Напишите имя исполнителя и название песни через /. '
                               'Например: Sia/Chandelier')
        bot.register_next_step_handler(msg, search_song)
    elif call.text == "Найти перевод песни":
        msg = bot.send_message(call.chat.id,
                               'Давайте найдем перевод песни! Напишите имя исполнителя и название песни через /. '
                               'Например: Sia/Chandelier')
        bot.register_next_step_handler(msg, search_translation)
    elif call.text == "Озвучь песню":
        msg = bot.send_message(call.chat.id,
                               'Давайте сделаем озвучку песни! Напишите имя исполнителя и название песни через /. '
                               'Например: Sia/Chandelier')
        bot.register_next_step_handler(msg, dub_song)

    elif call.text == "Заканчиваем":
        bot.send_message(call.chat.id, 'Хорошего дня! Когда захотите продолжнить нажмите на команду /start')

Прописываем функции search_translation и search_song и dub_song

In [ ]:
def search_song(msg):
    bot.send_message(msg.chat.id, f'Операция выполняется, пытаюсь найти текст для : {msg.text}')
    song_name = msg.text
    music_bot = MusicFinder(song_name)
    txt = music_bot.find_original_text()
    bot.send_message(msg.chat.id, txt)
    send_keyboard(msg, "Чем еще могу помочь?")


def search_translation(msg):
    bot.send_message(msg.chat.id, f'Операция выполняется, пытаюсь найти перевод для : {msg.text}')
    song_name = msg.text
    music_bot = MusicFinder(song_name)
    txt = music_bot.find_translation()
    bot.send_message(msg.chat.id, txt)
    send_keyboard(msg, "Чем еще могу помочь?")


def dub_song(msg):
    bot.send_message(msg.chat.id, f'Операция выполняется, пытаюсь найти исполнение для, '
                                  f'операция может занять некоторое время : {msg.text}')
    song_name = msg.text
    music_bot = MusicFinder(song_name)
    music_bot.dub_song()
    bot.send_audio(msg.chat.id, audio=open("bad_words_in_strange_order.mp3", 'rb'))
    send_keyboard(msg, "Чем еще могу помочь?")

В конце прописываем что будет выдавать бот, когда не понимает команду

In [ ]:
@bot.message_handler(content_types=['text'])
def handle_docs_audio(message):
    send_keyboard(message, text="Я не понимаю, выберите один из пунктов: ")


bot.polling(none_stop=True)